In [14]:
import pandas as pd 
import numpy as np
import os

In [15]:
# Loading the customer movement data for 5 days
directory_in_str = "./data/supermarket/"
directory = os.fsencode(directory_in_str)
data = []
for filename in os.listdir(directory):
    df_temp = pd.read_csv(directory_in_str+filename.decode('utf-8'),sep=';',index_col=0,parse_dates=True)
    df_temp['week_day'] = filename.decode('utf-8').split(".")[0]
    data.append(df_temp)
df = pd.concat(data)

In [16]:
df.head()

,customer_no,location,week_day
timestamp,,,
2019-09-06 07:00:00,1,dairy,friday
2019-09-06 07:00:00,2,drinks,friday
2019-09-06 07:00:00,3,fruit,friday
2019-09-06 07:01:00,2,checkout,friday
2019-09-06 07:01:00,4,drinks,friday


In [17]:
new_df= df.groupby(['customer_no','week_day'])['location'].resample('1T').ffill()

In [30]:
new_df.head(10)

customer_no  week_day  timestamp          
1            friday    2019-09-06 07:00:00       dairy
                       2019-09-06 07:01:00       dairy
                       2019-09-06 07:02:00       dairy
                       2019-09-06 07:03:00       dairy
                       2019-09-06 07:04:00      spices
                       2019-09-06 07:05:00    checkout
             monday    2019-09-02 07:03:00       dairy
                       2019-09-02 07:04:00       dairy
                       2019-09-02 07:05:00    checkout
             thursday  2019-09-05 07:00:00      drinks
Name: location, dtype: object

In [19]:
new_df.reset_index()

,customer_no,week_day,timestamp,location
0,1,friday,2019-09-06 07:00:00,dairy
1,1,friday,2019-09-06 07:01:00,dairy
2,1,friday,2019-09-06 07:02:00,dairy
3,1,friday,2019-09-06 07:03:00,dairy
4,1,friday,2019-09-06 07:04:00,spices
...,...,...,...,...
53898,1533,thursday,2019-09-05 21:48:00,dairy
53899,1534,thursday,2019-09-05 21:48:00,fruit
53900,1534,thursday,2019-09-05 21:49:00,checkout
53901,1535,thursday,2019-09-05 21:48:00,spices


In [20]:
#Transforming the dataframe for better implementation
transformed_df = new_df.reset_index().groupby(['customer_no','week_day'])[['timestamp','location']].agg(lambda x:list(x))
transformed_df

timestamp  \
customer_no week_day                                                       
1           friday     [2019-09-06 07:00:00, 2019-09-06 07:01:00, 201...   
            monday     [2019-09-02 07:03:00, 2019-09-02 07:04:00, 201...   
            thursday   [2019-09-05 07:00:00, 2019-09-05 07:01:00, 201...   
            tuesday    [2019-09-03 07:02:00, 2019-09-03 07:03:00, 201...   
            wednesday  [2019-09-04 07:00:00, 2019-09-04 07:01:00, 201...   
...                                                                  ...   
1531        wednesday  [2019-09-04 21:46:00, 2019-09-04 21:47:00, 201...   
1532        thursday                               [2019-09-05 21:48:00]   
1533        thursday                               [2019-09-05 21:48:00]   
1534        thursday          [2019-09-05 21:48:00, 2019-09-05 21:49:00]   
1535        thursday          [2019-09-05 21:48:00, 2019-09-05 21:49:00]   

                                                                location  
customer_no week_day                                                      
1           friday        [dairy, dairy, dairy, dairy, spices, checkout]  
            monday                              [dairy, dairy, checkout]  
            thursday                          [drinks, drinks, checkout]  
            tuesday    [fruit, fruit, fruit, drinks, drinks, drinks, ...  
            wednesday                           [fruit, fruit, checkout]  
...                                                                  ...  
1531        wednesday                 [drinks, drinks, drinks, checkout]  
1532        thursday                                             [dairy]  
1533        thursday                                             [dairy]  
1534        thursday                                   [fruit, checkout]  
1535        thursday                                  [spices, checkout]  

[7445 rows x 2 columns]

In [21]:
#creating transition matrix with before and after states
transition_df=pd.DataFrame(columns=['before','after'])
index = 0
for i in transformed_df.iterrows():
    customer_no = i[0][0]
    week_day = i[0][1]
    time_stamp = i[1][0]
    sections = i[1][1]
    sections.insert(0,'entrance')
    entrance_time = pd.to_datetime(time_stamp[0]) + pd.to_timedelta(-1, unit='m')
    time_stamp.insert(0,entrance_time)
    for j in range(len(sections)):
        if (j+1)<len(sections):
            transition_df.loc[index]=(sections[j],sections[j+1])
            index += 1

In [22]:
transition_df.head(10)

,before,after
0,entrance,dairy
1,dairy,dairy
2,dairy,dairy
3,dairy,dairy
4,dairy,spices
5,spices,checkout
6,entrance,dairy
7,dairy,dairy
8,dairy,checkout
9,entrance,drinks


In [24]:
#creating transition probability matrix
transition_probability_matrix = pd.crosstab(transition_df['before'], transition_df['after'], normalize=0)
transition_probability_matrix

after,checkout,dairy,drinks,fruit,spices
before,,,,,
dairy,0.102759,0.737331,0.058615,0.049887,0.051408
drinks,0.215549,0.010901,0.598622,0.087927,0.087001
entrance,0.000000,0.287576,0.153526,0.377435,0.181464
fruit,0.201165,0.095977,0.054878,0.597276,0.050705
spices,0.150008,0.193368,0.163239,0.091025,0.402359


In [25]:
transition_probability_matrix['entrance']=[0,0,0,0,0]
transition_probability_matrix.loc['checkout'] = [1,0,0,0,0,0]

In [26]:
# ordering the columns inside
transition_probability_matrix = pd.DataFrame(transition_probability_matrix,columns=['entrance','dairy','drinks','fruit','spices','checkout'],index=['entrance','dairy','drinks','fruit','spices','checkout'])
#print(transition_probability_matrix)


In [27]:
transition_probability_matrix 

,entrance,dairy,drinks,fruit,spices,checkout
entrance,0,0.287576,0.153526,0.377435,0.181464,0.000000
dairy,0,0.737331,0.058615,0.049887,0.051408,0.102759
drinks,0,0.010901,0.598622,0.087927,0.087001,0.215549
fruit,0,0.095977,0.054878,0.597276,0.050705,0.201165
spices,0,0.193368,0.163239,0.091025,0.402359,0.150008
checkout,0,0.000000,0.000000,0.000000,0.000000,1.000000


In [29]:
assert all(transition_probability_matrix.sum(axis=1) > 0.999)

In [28]:
transition_probability_matrix.to_csv('transition_matrix.csv')